### Урок 4. Парсинг HTML. XPath

* Выберите веб-сайт с табличными данными, который вас интересует.
* Напишите код Python, использующий библиотеку requests для отправки HTTP GET-запроса на сайт и получения HTML-содержимого страницы.
* Выполните парсинг содержимого HTML с помощью библиотеки lxml, чтобы извлечь данные из таблицы.
* Сохраните извлеченные данные в CSV-файл с помощью модуля csv.

Ваш код должен включать следующее:

- Строку агента пользователя в заголовке HTTP-запроса, чтобы имитировать веб-браузер и избежать блокировки сервером.
- Выражения XPath для выбора элементов данных таблицы и извлечения их содержимого.
- Обработка ошибок для случаев, когда данные не имеют ожидаемого формата.
- Комментарии для объяснения цели и логики кода.

Целью данной работы будет попытка получить данные по турнирам настольного тенниса от Лига-Про.

1. Пытаемся получить ответ с get-запроса по интересующему нас дню
2. Получить информацию по этому дню о предстоящих турнирах со списком участников
3. Сформировать из полученных данных датафрейм
4. Сохранить данных в табличном формате в файле csv

In [11]:
#!pip install lxml

In [12]:
# импортируем нужные библиотеки
from lxml import html
import requests
from pprint import pprint
import csv
import pandas as pd

In [13]:
url='https://tt.sport-liga.pro/'

# задаем user-agent
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }
# в params надо указать интересующую нас дату в формате YYYY:MM:DD
param = {
    'year': '2024',
    'month': '05',
    'day': '2'
}

url_tour = 'tours/'
session = requests.Session()

In [14]:
# просьба ввести данные пользователя из консоли (В РАЗРАБОТКЕ)

# year, month, day = input('Введите дату в формате YYYY:MM:DD: ').split('.')
# params = {
#     'year': year,
#     'month': month,
#     'day': day
# }

In [15]:
# делаем get-запрос
response = session.get(url+url_tour, params=param, headers=header)
# проверяем статус-код
if response.status_code < 400:
    # если да, печатаем статус код и представляем ответ в виде дерева html
    print(response.status_code)
    tree = html.fromstring(response.text)
else:
    print('Что-то пошло не так')

200


Необходимые данные находятся на странице в нижней таблице. Также для сбора фамилий участников турнира нам нужно переходить на его страницу (ее можно "достать" из таблицы.)

In [16]:
# находим нужную таблицу на странице
table = tree.xpath('//table[@class="table"]//tr')
# находим ссылки на переход к просмотру турнира
ref = tree.xpath('.//td[@class="tournament-name"]/a/@href')

Для объединения всех данных в один список будем проходиться циклом for сразу по двум спискам, для этого объединим их функцией zip

In [17]:
tournaments = []
for row, item in zip(table[1:], ref):
    # сделаем get-запрос на страницу турнира
    res = session.get(url+item, headers=header)
    # и получим дерево из его ответа
    tour = html.fromstring(res.text)
    # создадим словарик для каждого турнира
    tours_info = {}

    # из даты нам нужно только время начало турнира
    date = row.xpath('.//td[@class="tournament-date"]/text()')
    date = [el.split()[-1] for el in date][0]

    # из названия турнира мы возьмем
    tours = row.xpath('.//td[@class="tournament-name"]/a/text()')
    # название стола
    tables = [el.split()[1][:-1] for el in tours][0]
    # уровень лиги
    liga = [el.split()[3] for el in tours][0]

    # участников турнира получим из страницы турнира
    parties = tour.xpath('//td[@class="nowrap"]/a/text()')

    tours_info['date'] = date
    tours_info['tables'] = tables
    tours_info['liga'] = liga
    tours_info['parties'] = parties
    
    # добавим каждый словарь в список
    tournaments.append(tours_info)


In [18]:
pprint(tournaments)

[{'date': '23:45',
  'liga': '350-400',
  'parties': ['Мареев Н',
              'Кугурушев Д',
              'Немашкало В',
              'Кадулин М',
              'Янковский А'],
  'tables': 'А6'},
 {'date': '23:45',
  'liga': '550-600',
  'parties': ['Лыфенко Н',
              'Солдатов И',
              'Лопатин С',
              'Еремин В',
              'Турабеков Т'],
  'tables': 'А4'},
 {'date': '23:30',
  'liga': '450-500',
  'parties': ['Пандур И',
              'Тарасенков Н',
              'Зеленский Р',
              'Зюганов О',
              'Корнеев Д'],
  'tables': 'А5'},
 {'date': '20:00',
  'liga': '550-600.',
  'parties': ['Токаревский А', 'Логунов А', 'Мартычан И', 'Сидорков А'],
  'tables': 'А15'},
 {'date': '20:00',
  'liga': '800-900',
  'parties': ['Пищук Д', 'Рахматов А', 'Инназаров А', 'Чернов А'],
  'tables': 'А9'},
 {'date': '20:00',
  'liga': '250-300',
  'parties': ['Смирнов М', 'Кондратьев Г', 'Егоров П', 'Рохлин И'],
  'tables': 'А3'},
 {'date': '19:45'

Запишем полученную информацию в csv-файл

In [19]:
with open('./tournaments.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=tournaments[0].keys())
    writer.writeheader
    for row in tournaments:
        writer.writerow(row)
print('Данные успешно записаны')

Данные успешно записаны


Для наглядности представим наш список в виде датафрейма

In [20]:
df = pd.DataFrame(tournaments)
df

,date,tables,liga,parties
0,23:45,А6,350-400,"[Мареев Н, Кугурушев Д, Немашкало В, Кадулин М..."
1,23:45,А4,550-600,"[Лыфенко Н, Солдатов И, Лопатин С, Еремин В, Т..."
2,23:30,А5,450-500,"[Пандур И, Тарасенков Н, Зеленский Р, Зюганов ..."
3,20:00,А15,550-600.,"[Токаревский А, Логунов А, Мартычан И, Сидорко..."
4,20:00,А9,800-900,"[Пищук Д, Рахматов А, Инназаров А, Чернов А]"
5,20:00,А3,250-300,"[Смирнов М, Кондратьев Г, Егоров П, Рохлин И]"
6,19:45,А6,500-550,"[Вашков А, Илюхин Е, Ковалев И, Пучинин А]"
7,19:45,А4,550-600,"[Мошков И, Артемов А, Мехасюк И, Никулин С]"
8,19:30,А5,400-450,"[Трошкин А, Круглов А, Вологжанин В, Кузьмин С]"
9,16:00,А3,300-350,"[Ланин С, Ханевский А, Давлитшин И, Голованов Г]"


Вывод: В результате проделанной работы мы получаем датафрейм, содержащий нужную информацию. Этот датафрейм планируется использовать для подбора турнира с более предпочтительными игроками.

Также планируется исследовать каждый турнир на предмет статистики личных встреч между игроками